In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from akorn import *
from flh import *
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [2]:
%load_ext autoreload
%autoreload 2
# import numpy as np
# import math
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr, data



utils = importr('utils')
base = importr('base')
glmgen = importr('glmgen')
%matplotlib qt

std = 0.3
var = std**2

# mse_akorn = []
# mse_tf = []



def doppler(x, epsilon):
    return np.sin(2 * np.pi * (1 + epsilon) / (x + epsilon))

from util import piecewise_linear

print("Done!")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done!


In [15]:
mse_akorn = []
mse_addle = []
mse_tf = []
mse_tf_dof = []
# Ts = [100, 5000, 100_000, 200_000, 500_000]
# Ts = [50, 100, 200, 500, 1000, 2500, 5000]
Ts = [50, 100, 200, 500, 1000, 2500, 5000]



# Ts = [100]
runs_per_T = 20

from util import *

akorn_fv = None
dof_fv = None
tf_fv = None
addle_fv = None
    
for T in Ts:
    print("T = ", T)
    X = np.linspace(0, T, T)/T 
    Xr = robjects.FloatVector(X)



    G = doppler(abs(1 - X), 0.38)

    mses_T_tf = []
    mses_T_tf_dof = []
    mses_T_akorn = []
    mses_T_addle = []

    for l in range(runs_per_T):
        print("Run number " + str(l))
        Y = (G + np.random.normal(0, std, T))

        ak = AKORN(X, Y, var)
        ak.train()
        fv = ak.preds
        
        online = FLH(X, Y, T, lr = 1/(2*np.max(np.abs(Y))**2))
        online.run()
        fv_addle = online.predictions
        
        mses_T_addle.append(((fv_addle - G)**2).mean())


        mses_T_akorn.append(((fv - G)**2).mean())


        Yr = robjects.FloatVector(Y)
        tf = glmgen.trendfilter(Xr, Yr, k = 1)
        rcode1 = "predict(%s)" %(tf.r_repr())
        tf_fit = np.array(robjects.r(rcode1)).T

        best_mse = float('inf')
        best_mse_est = float('inf')
        best_mse_dof = float('inf')
        best_tf_fit = None
        best_tf_fit_dof = None
        for t in tf_fit:
            tf_mse = ((t - G)**2).mean() #Oracle risk
            tf_mse_est = (np.sum(((t - Y)**2)) + 2*var*count_linear_pieces(t)) #Stein risk estimate
            if tf_mse < best_mse:
                best_mse = tf_mse
                best_tf_fit = t
            if tf_mse_est < best_mse_est:
                best_mse_est = tf_mse_est
                best_mse_dof = tf_mse
                best_tf_fit_dof = t
                
        mses_T_tf.append(((best_tf_fit - G)**2).mean())
        mses_T_tf_dof.append(((best_tf_fit_dof - G)**2).mean())
        
        akorn_fv = fv
        addle_fv = fv_addle
        tf_dof_fv = best_tf_fit_dof
        tf_fv = best_tf_fit

    #     Z_tf.append(best_tf_fit)
    mse_tf.append(mses_T_tf)
    mse_tf_dof.append(mses_T_tf_dof)
    mse_addle.append(mses_T_addle)
    mse_akorn.append(mses_T_akorn)
    

avg_mses_akorn = [np.array(mse_T).mean() for mse_T in mse_akorn]
avg_mses_addle = [np.array(mse_T).mean() for mse_T in mse_addle]
avg_mses_tf = [np.array(mse_T).mean() for mse_T in mse_tf]
avg_mses_tf_dof = [np.array(mse_T).mean() for mse_T in mse_tf_dof]
print("Done!")

T =  50
Run number 0
lr =  0.06540133987389826
run
0
Run number 1
lr =  0.0671002079722877
run
0
Run number 2
lr =  0.034070041716838514
run
0
Run number 3
lr =  0.07050286703279911
run
0
Run number 4
lr =  0.06630845474622049
run
0
Run number 5
lr =  0.06137381354030101
run
0
Run number 6
lr =  0.09883501478391288
run
0
Run number 7
lr =  0.03540224043330935
run
0
Run number 8
lr =  0.04858388484143189
run
0
Run number 9
lr =  0.04782642385060567
run
0
Run number 10
lr =  0.07543521283503012
run
0
Run number 11
lr =  0.05410168774057989
run
0
Run number 12
lr =  0.0388482111087155
run
0
Run number 13
lr =  0.062467334459438036
run
0
Run number 14
lr =  0.060537715846552764
run
0
Run number 15
lr =  0.05757417718681613
run
0
Run number 16
lr =  0.056886887357712794
run
0
Run number 17
lr =  0.06786204684730517
run
0
Run number 18
lr =  0.04735474647272902
run
0
Run number 19
lr =  0.06588164763604748
run
0
T =  100
Run number 0
lr =  0.061824645250651074
run
0
Run number 1
lr =  0.0449

In [16]:
from sklearn.linear_model import LinearRegression
akorn_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_akorn))
addle_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(avg_mses_addle))
tf_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf)))
tf_dof_ref = LinearRegression().fit(np.log(np.array(Ts)).reshape(-1, 1), np.log(np.array(avg_mses_tf_dof)))

In [17]:
def quick_round_str(z):
    return str(round(z, 5))

plt.clf()
from util import subplots_centered
T = Ts[-1]
X = np.linspace(0, T, T)/T 
G = doppler(abs(1 - X), 0.38)

f, ax = plt.subplots(1, 2)
# f, ax = subplots_
ax0 = ax[0]
ax1 = ax[1]
ax1.plot(range(T), G, color = "black")
ax1.plot(range(T), tf_fv)
ax1.plot(range(T), Y, alpha = 0.15, color = "black")
# ax1.plot(range(T), tf_fv_dof)
ax1.plot(range(T), akorn_fv)
ax1.plot(range(T), addle_fv, color = "green")

ax1.yaxis.set_visible(False)

# f.suptitle("Rates for PW Linear Function")
ax0.set_xlabel("n")
ax0.set_ylabel("MSE")

# ax0.set_ylim([10**(-4), 10**(-1)])
# ax1.set_ylim([10**(-4), 10**(0)])

ax0.grid(True)



ax0.loglog(Ts, avg_mses_tf, marker = 'o', label = "oracle trend filtering, rate = " +  quick_round_str(tf_ref.coef_[0]))
ax0.loglog(Ts, avg_mses_addle, marker = 'o', label = "addle, rate = " + quick_round_str(addle_ref.coef_[0]), color = "green")
ax0.loglog(Ts, avg_mses_tf_dof, marker = 'o', label = "tf dof, rate = " + quick_round_str(tf_dof_ref.coef_[0]), color = "blue")
ax0.loglog(Ts, avg_mses_akorn, marker = 'o', label = "akorn, rate = " + quick_round_str(akorn_ref.coef_[0]), color = "orange")

f.legend()

In [ ]:
mses_akorn